In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# pip install category_encoders

In [ ]:
# pip install catboost

In [ ]:
# pip install optuna

In [2]:
# Library
import os
import random
import pickle
import gc
import warnings
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import KMeans, MiniBatchKMeans
from category_encoders import TargetEncoder
from sklearn.preprocessing import (
    StandardScaler, PowerTransformer, OrdinalEncoder,
    OneHotEncoder, FunctionTransformer, PolynomialFeatures, LabelEncoder,
)
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import (
    LogisticRegression, LinearRegression, Ridge, Lasso,
    SGDRegressor, ElasticNet
)
from sklearn.model_selection import (
    train_test_split, cross_val_score, cross_validate,
    GridSearchCV, KFold, cross_val_predict
)
from sklearn.metrics import (
    roc_auc_score, mean_squared_error, make_scorer, accuracy_score, log_loss
)
from sklearn import set_config, datasets
from catboost import (
    CatBoostRegressor, CatBoostClassifier,
)
import category_encoders as ce
from sklearn.pipeline import (
    Pipeline, FeatureUnion, make_pipeline
)
from sklearn.ensemble import (
    RandomForestClassifier, StackingClassifier, StackingRegressor,
    GradientBoostingRegressor, VotingClassifier, VotingRegressor,
    HistGradientBoostingRegressor, GradientBoostingClassifier,
    BaggingClassifier, AdaBoostClassifier, RandomForestRegressor,ExtraTreesRegressor
)
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.svm import SVC, SVR, LinearSVC
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons

import xgboost as xgb
import lightgbm as lgb
import re
import math
import optuna
from pandas.errors import DataError
from scipy.stats import zscore

%matplotlib inline

warnings.filterwarnings("ignore")

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(42)

In [4]:
# 데이터 로드
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_sessionID = test.sessionID

# train = pd.read_csv("/content/drive/MyDrive/3-2/머신러닝/dacon/웹 로그 기반 조회수 예측/train.csv")
# test = pd.read_csv("/content/drive/MyDrive/3-2/머신러닝/dacon/웹 로그 기반 조회수 예측/test.csv")
# target = train.TARGET
# test_sessionID = test.sessionID

In [5]:
import sweetviz as sv
# Sweetviz 보고서 생성

report_train = sv.analyze(train)
report_train.show_html("report_train.html")

# report_test = sv.analyze(test)
# report_test.show_html("reprot_test.html")

                                             |          | [  0%]   00:00 -> (? left)

Report report_train.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
train.head()

,sessionID,userID,TARGET,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,17.0,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,3.0,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,1.0,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,1.0,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,1.0,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000


In [7]:
#전처리
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    df = pd.concat([train,test]).reset_index(drop = True)

    #userID : 앞 부분 날리기
    df['userID'] = df['userID'].str.replace('USER_','')
    df['userID'].astype('int64')

    # browser : 상위 20개 제외 other 변경
    browser_list = df['browser'].tolist()
    browser_list = list(set(browser_list))

    top20_list = ['Chrome', 'Samsung Internet', 'Firefox', 'Opera',
       'Safari (in-app)', 'Edge', 'Safari', 'Android Webview',
       'UC Browser', 'Internet Explorer', 'Opera Mini', 'BlackBerry',
       'Android Browser', 'Amazon Silk', 'Coc Coc', 'YaBrowser',
       'Nintendo Browser', 'Nokia Browser', 'MRCHROME', 'Puffin']
    other_set = set(browser_list) - set(top20_list)
    replace_dict = {item: 'Other' for item in other_set}
    df['browser'] = df['browser'].replace(replace_dict)

    df['quality_duration1'] = df['quality'] * df['duration']
    df['quality_duration2'] = df['quality'] + df['duration']
    df['quality_duration3'] = df['quality'] - df['duration']

    df['keyword'].fillna('NAN', inplace=True)
    df['referral_path'].fillna('NAN', inplace=True)

    train = df[~df["TARGET"].isnull()].sort_values("sessionID").reset_index(drop = True)
    test = df[df["TARGET"].isnull()].sort_values("sessionID").reset_index(drop=True)
    test = test.drop(columns=["TARGET"])

    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

train_pre, test_pre = pre_all(train, test)

전처리 전 train 크기 : (252289, 19)
전처리 전 test 크기 : (79786, 18)
=================전처리 중=================
전처리 후 train 크기 : (252289, 24)
전처리 후 test 크기 : (79786, 23)


### Encoding

In [8]:
from catboost import CatBoostRegressor, Pool
categorical_features = [
"browser",
"OS",
"device",
"continent",
"subcontinent",
"country",
"traffic_source",
"traffic_medium",
"keyword",
"referral_path",
]
for i in categorical_features:
    train_pre[i] = train_pre[i].astype('category')
    test_pre[i] = test_pre[i].astype('category')

In [9]:
x_train = train_pre.drop(['TARGET', "sessionID"], axis=1)
target = train_pre['TARGET']

In [10]:
# X_train, X_val, y_train, y_val = train_test_split(x_train, target, test_size=0.3, random_state=42)

In [11]:
# # 훈련 데이터와 검증 데이터로 분할


# # CatBoost 데이터셋 생성
# train_pool = Pool(X_train, label=y_train, cat_features=categorical_features)
# val_pool = Pool(X_val, label=y_val,cat_features=categorical_features)

# # CatBoostRegressor 모델 정의
# def objective(trial):
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 1000),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
#         'random_state': 42
#     }

#     model = CatBoostRegressor(**params)
#     model.fit(train_pool, eval_set=val_pool, verbose=100)

#     # 검증 데이터에 대한 성능 지표 계산 (원하는 지표로 변경 필요)
#     val_metric = model.get_best_score()['validation']['RMSE']

#     # 각 반복에서의 검증 성능 출력
#     print(f"Iteration {trial.number}: Validation RMSE - {val_metric}")

#     return val_metric

# # Optuna 최적화 실행
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=20)

# # 최적화된 하이퍼파라미터 확인
# best_params = study.best_params
# print(f"Best Hyperparameters: {best_params}")

# best_model = CatBoostRegressor(**best_params)
# best_model.fit(train_pool, eval_set=val_pool, verbose=20)

# # 최적 모델의 검증 성능 출력
# best_val_metric = best_model.get_best_score()['validation']['RMSE']
# print(f"Best Model Validation RMSE: {best_val_metric}")

# # 최적의 하이퍼파라미터 출력
# print(f"Best Hyperparameters: {best_params}")


### 검증

## model

In [12]:
pool_train = Pool(data=x_train, label=target, cat_features=categorical_features)

In [13]:
clf = CatBoostRegressor(iterations=692, depth=10, learning_rate=0.058209774105192055)
clf.fit(pool_train)

0:	learn: 5.2950101	total: 387ms	remaining: 4m 27s
1:	learn: 5.0915837	total: 634ms	remaining: 3m 38s
2:	learn: 4.9048686	total: 874ms	remaining: 3m 20s
3:	learn: 4.7319010	total: 1.11s	remaining: 3m 11s
4:	learn: 4.5714835	total: 1.32s	remaining: 3m 1s
5:	learn: 4.4242051	total: 1.57s	remaining: 2m 59s
6:	learn: 4.2872265	total: 1.84s	remaining: 3m
7:	learn: 4.1604856	total: 2.08s	remaining: 2m 58s
8:	learn: 4.0459132	total: 2.31s	remaining: 2m 55s
9:	learn: 3.9398446	total: 2.57s	remaining: 2m 55s
10:	learn: 3.8399238	total: 2.82s	remaining: 2m 54s
11:	learn: 3.7497257	total: 3.03s	remaining: 2m 51s
12:	learn: 3.6676212	total: 3.26s	remaining: 2m 50s
13:	learn: 3.5919264	total: 3.49s	remaining: 2m 49s
14:	learn: 3.5226290	total: 3.71s	remaining: 2m 47s
15:	learn: 3.4557364	total: 3.96s	remaining: 2m 47s
16:	learn: 3.3970335	total: 4.2s	remaining: 2m 46s
17:	learn: 3.3437850	total: 4.44s	remaining: 2m 46s
18:	learn: 3.2917884	total: 4.67s	remaining: 2m 45s
19:	learn: 3.2446365	total: 

In [14]:
test_pool = Pool(data=test_pre.drop(columns = 'sessionID'), cat_features=categorical_features)
pred = clf.predict(test_pool)
pred = [0 if i < 0 else i for i in pred]

## submission

In [15]:
t = pd.Timestamp.now()
fname = f"weblog_submission_{t.month:02}월{t.day:02}일{t.hour:02}시{t.minute:02}분.csv"
pd.DataFrame({'sessionID': test_sessionID, 'TARGET': pred}).to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'weblog_submission_02월20일22시27분.csv' is ready to submit.
